# Simple Code-Shortening Idioms #

A collection of some very simple techniques for performing common tasks, that hopefully you will find useful.

## Conditionally Deleting A Dictionary Entry ##

You may know that you can delete a dictionary entry with

    del dict[key]

but this raises a `KeyError` exception if `key` is not present in the dictionary. So how do you delete an entry _only if it is present_ ? This

    if key in dict :
        del dict[key]
    #end if

seems unwieldy. But there is a shorter way. The `dict` class defines [a pop method](https://docs.python.org/3/library/stdtypes.html#mapping-types-dict) that removes and returns an entry with a specified key; this call also raises `KeyError` if the key is not present, unless you specify a default value that is returned in this case. So, to delete an entry if present and avoid any error from its absence, pass any dummy default value (e.g. `None`) and ignore the result:

    dict.pop(key, None)

## Testing Mask Bits ##

You may not have immediately noticed from reading the Python language spec, but [binary bitwise operators have higher precedence than comparisons](https://docs.python.org/3/reference/expressions.html#binary-bitwise-operations). This differs from C and most other languages that follow C conventions. You may think this could easily trip up someone transliterating expressions from C or other such languages to Python, but I am not aware of any real-world situation where this could be a problem.

What it _does_ mean is that you can write bitmask tests with fewer parentheses. For example, in C you have to write things like

    («flags» & «mask») != 0
    («flags» & «mask1») == «mask2»

but in Python these parentheses are unnecessary:

    «flags» & «mask» != 0
    «flags» & «mask1» == «mask2»
